# Project Documentation: AI-Driven Customer Engagement & RFM Analytics Platform

## 1. Executive Summary
This project implements an end-to-end data engineering and data science pipeline designed to automate personalized marketing communications. By integrating **ETL processes**, **RFM Analysis**, and **Generative AI**, the system transforms raw transaction data into actionable, personalized email drafts for customer retention and loyalty programs.

## 2. Data Source & Licensing

* **Dataset Source:** The project utilizes the **"An Online Shop Business"** dataset hosted on Kaggle (`gabrielramos87/an-online-shop-business`).
* **Data Description:** The dataset contains e-commerce transaction logs, including fields such as `TransactionNo`, `Date`, `ProductNo`, `ProductName`, `Price`, `Quantity`, `CustomerNo`, and `Country`.
* **Licensing:** The dataset is publicly available under the **CC0: Public Domain** license (or standard Kaggle Dataset license), allowing for unrestricted use for analytical, educational, and commercial purposes without mandatory attribution.

## 3. Technology Stack

* **Language:** Python 3.13+
* **Orchestration & Data Manipulation:** Pandas, NumPy
* **Database:** PostgreSQL (hosted locally)
* **ORM & Connection:** SQLAlchemy, Psycopg2
* **GenAI Inference:** Groq API (LPU Inference Engine)
* **Model:** `openai/gpt-oss-120b` (Open Weights Model)
* **Environment Management:** Dotenv (`.env`) for secure credential storage

## 4. Methodology & Architecture

The pipeline is divided into three distinct operational phases:

### Phase I: ETL Pipeline (Extract, Transform, Load)
*Reference: `01_ETL_pipeline_and_database_setup.ipynb`, `database_methods.py`*

1.  **Ingestion:** The dataset is downloaded programmatically using the `kagglehub` library.
2.  **Transformation & Cleaning:**
    * **Standardization:** Column names are converted from `CamelCase` to `snake_case` (e.g., `ProductNo` $\to$ `product_id`).
    * **Type Casting:** Optimization of data types (e.g., `category` for low-cardinality columns, `float32` for prices) to reduce memory usage.
    * **Data Hygiene:** Rows with `NaN` values in critical fields (Customer ID) are removed. Negative quantities (indicating returns) are filtered out to ensure analysis focuses only on valid sales.
3.  **Database Loading:**
    * The system creates a PostgreSQL database connection.
    * **Optimization:** A custom high-performance insertion method (`psql_insert_copy`) is implemented in `database_methods.py`. This utilizes the PostgreSQL `COPY FROM STDIN` command instead of standard `INSERT` statements, significantly reducing load time for large datasets.
    * **Indexing:** An index is created on `customer_id` to optimize downstream SQL query performance.

### Phase II: RFM Analysis & Segmentation
*Reference: `02_RFM_segmentation_and_GenAI_agent.ipynb`*

1.  **Feature Engineering (SQL-Based):**
    Instead of processing raw data in Python, an optimized SQL query aggregates data to calculate **RFM** metrics directly:
    * **Recency:** Days since the last purchase.
    * **Frequency:** Count of distinct orders.
    * **Monetary:** Total sum of revenue (`quantity * price`).
    * **Churn Indicator:** Customers inactive for $\ge$ 90 days are flagged.
2.  **Segmentation Logic:**
    Customers are categorized based on dynamic thresholds (80th percentile of the dataset):
    * **VIP Loyalty:** Active customers with high Monetary or Frequency scores.
    * **Churn Recovery:** Inactive customers (Recency $\ge$ 90 days).
    * **Standard Promo:** Active customers who do not yet meet VIP criteria.

### Phase III: Generative AI Agent
*Reference: `ai_agent_methods.py`*

1.  **Inference Provider:** The system utilizes the **Groq API** to leverage LPU (Language Processing Unit) architecture for ultra-low latency text generation.
2.  **Model Selection:** The project utilizes **`openai/gpt-oss-120b`**, a powerful open-weights model available on Groq.
3.  **Prompt Engineering:**
    * **Dynamic Prompts:** Specific instructions are injected based on the customer's segment (e.g., "We miss you" tone for churned customers vs. "Exclusive offer" for VIPs).
    * **Strict Constraints:** The system uses "Negative Constraints" (e.g., *PROHIBITED: Do NOT use placeholders*) and "Strict Formatting Rules" to ensure the output is production-ready.
4.  **Sequential Processing:**
    * To respect API rate limits (RPM/TPM), the system processes customers sequentially using `pandas.apply()` with a `time.sleep(1)` interval. This ensures stability and prevents `429 Too Many Requests` errors.

## 5. Outputs & Business Intelligence Integration

* **AI Content Review:** To facilitate manual inspection and quality assurance, the generated email drafts and corresponding `customer_id`s are exported to a **CSV file** (`marketing_campaign_drafts.csv`). This allows project reviewers to easily analyze the output of the Generative AI agent without processing the entire dataset.

* **Data Serialization:** The cleaned transactional data from the initial ETL phase (prior to SQL ingestion) is preserved in **Parquet format** (`e_commerce_order_details.parquet`) to ensure high-performance I/O and type consistency.

* **Visualization (Power BI):** This Parquet file serves as the primary data source for **Microsoft Power BI**. The dashboarding phase utilizes this cleaned transactional dataset to demonstrate comprehensive BI skills, including data modeling, DAX calculations, and visual storytelling, providing insights into general sales performance.

